In [ ]:
pip install selenium

In [4]:
import numpy as np
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re
import json

In [5]:
PATH = "C:\\Users\\Victor\\Downloads\\chromedriver_win32\\chromedriver" # Change this file path to match the chromedriver location on your device!
driver = webdriver.Chrome(PATH)

C:\Users\Victor\AppData\Local\Temp/ipykernel_26016/1940386753.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [6]:
search_terms = [
  #  'beef stew',
  #  'salad',
  #  'pasta',
  #  'steak',
  #   'fish',
  #   'smoothie',
  #   'meatballs',
  #   'chicken',
  #  'burger',
  #  'cookie',
  #  'eggs',
  #  'bread',
  #  'sandwhich',
  #  'taco',
  #  'meatloaf',
    'pizza',
    'cake',
    'fried rice',
    'chili',
    'curry',
    'dip',
    'burrito',
    'gumbo',
    'chicken noodle soup',
  #  'dessert',
]

data = []


# Replace XXX with the search term and offset=i
SEARCH_URL = "https://www.allrecipes.com/search?XXX=XXX&q=XXX&offset=0"


In [7]:
def clean_field(entry_data, s):
    l = s.rsplit(' ', 1)
    entry_data[l[0]] = int(''.join(c for c in l[1] if c.isdigit()))
    # string = r'data-doc-id="[0-9]+'
    # match = re.search(string, page.text)

# Search for search_term and retrieve at most n recipes
def scrape(search_term, n):
    
    i = 0
    while True:
        
        i += 1
        if i == n:
            break
        
        try:
            
            # Holds the data on the current recipe entry
            entry_data = {}
            
            entry_data['search_term'] = search_term
            
            # Modify search url
            search_term = search_term.replace(" ", "%20")
            url = SEARCH_URL.replace("0", str(i))
            url = url.replace("XXX", search_term)

            driver.get(url)
            # //*[@id="mntl-card-list-items_1-0"]
            link = driver.find_element(By.XPATH, '//*[@id="mntl-card-list-items_1-0"]')

            # Make sure the link is actually a recipe and not a dumb blogpost smh
            u = link.get_attribute('href')      
            if "/recipe/" not in u:
                n += 1
                continue
            print(u)
            
            # Click on link
            link.click()
            
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//*[@id="article-heading_1-0"]')))
            # Extract recipe name and rating
            entry_data['name'] = driver.find_element(By.XPATH, '//*[@id="article-heading_1-0"]').text
            try:
                entry_data['rating'] = driver.find_element(By.XPATH, '//*[@id="mntl-recipe-review-bar__rating_1-0"]').text
            except Exception as e:
                entry_data['rating'] = None
                
            # Click on "show nutrition info" button
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//*[@id="mntl-nutrition-facts-label_1-0"]/button'))).click()

            # Extract the nutrition data
            entry_data['calories'] = driver.find_element(By.XPATH, '//*[@id="mntl-nutrition-facts-label_1-0"]/div/div/table/thead/tr[3]/th/span[2]').text
            entry_data['servings'] = driver.find_element(By.XPATH, '//*[@id="mntl-nutrition-facts-label_1-0"]/div/div/table/thead/tr[2]/th/span[2]').text
            
            NUTRITION_FIELD =  '//*[@id="mntl-nutrition-facts-label_1-0"]/div/div/table/tbody/tr[XXX]/td[1]'
            
            j = 2
            while True:
                try:
                    nutrition_field = NUTRITION_FIELD.replace('XXX', str(j))
                    clean_field(entry_data, driver.find_element(By.XPATH, nutrition_field).text)
                except Exception as e:
                    break
                j = j+1
            
            print(entry_data)
            data.append(entry_data)
            
        except Exception as e:
            print(e)
            print("Ran into an error. Proceeding...")
            continue
        

In [ ]:
for s in search_terms:
    print("Scraping for " + s)
    scrape(s, 500)

Scraping for pizza
https://www.allrecipes.com/recipe/255299/jans-copycat-version-of-pizza-huts-stuffed-crust-pizza/
{'search_term': 'pizza', 'name': "Jan's Copycat Version of Pizza Hut's Stuffed Crust Pizza", 'rating': '5.0', 'calories': '87', 'servings': '24', 'Total Fat': 6, 'Saturated Fat': 3, 'Cholesterol': 14, 'Sodium': 357, 'Total Carbohydrate': 2, 'Dietary Fiber': 0, 'Total Sugars': 1, 'Protein': 5, 'Calcium': 103, 'Iron': 0, 'Potassium': 32}
https://www.allrecipes.com/recipe/235563/ritz-white-pizza-meatball-dip-created-by-lombardis-pizza/
{'search_term': 'pizza', 'name': "RITZ White Pizza Meatball Dip, created by Lombardi's Pizza", 'rating': '4.5', 'calories': '168', 'servings': '28', 'Total Fat': 12, 'Saturated Fat': 4, 'Cholesterol': 20, 'Sodium': 237, 'Total Carbohydrate': 11, 'Dietary Fiber': 0, 'Total Sugars': 1, 'Protein': 4, 'Vitamin C': 1, 'Calcium': 48, 'Iron': 0, 'Potassium': 53}
https://www.allrecipes.com/recipe/20171/quick-and-easy-pizza-crust/
{'search_term': 'pizz

https://www.allrecipes.com/recipe/246552/brooklyn-style-pizza-dough/
{'search_term': 'pizza', 'name': 'Brooklyn Style Pizza Dough', 'rating': '4.0', 'calories': '34', 'servings': '8', 'Total Fat': 3, 'Saturated Fat': 1, 'Sodium': 147, 'Total Carbohydrate': 1, 'Dietary Fiber': 0, 'Protein': 1, 'Calcium': 2, 'Iron': 0, 'Potassium': 30}
https://www.allrecipes.com/recipe/232960/california-grilled-pizza/
{'search_term': 'pizza', 'name': 'California Grilled Pizza', 'rating': '4.6', 'calories': '224', 'servings': '6', 'Total Fat': 9, 'Saturated Fat': 3, 'Cholesterol': 11, 'Sodium': 361, 'Total Carbohydrate': 27, 'Dietary Fiber': 2, 'Total Sugars': 2, 'Protein': 10, 'Vitamin C': 5, 'Calcium': 81, 'Iron': 0, 'Potassium': 84}
https://www.allrecipes.com/recipe/222197/authentic-pizza-margherita/
{'search_term': 'pizza', 'name': 'Authentic Pizza Margherita', 'rating': '4.5', 'calories': '966', 'servings': '4', 'Total Fat': 37, 'Saturated Fat': 16, 'Cholesterol': 90, 'Sodium': 2187, 'Total Carbohydr

https://www.allrecipes.com/recipe/242467/almond-flour-pizza-crust/
{'search_term': 'pizza', 'name': 'Almond Flour Pizza Crust', 'rating': '4.1', 'calories': '261', 'servings': '8', 'Total Fat': 23, 'Saturated Fat': 3, 'Cholesterol': 70, 'Sodium': 176, 'Total Carbohydrate': 8, 'Dietary Fiber': 5, 'Total Sugars': 0, 'Protein': 10, 'Vitamin C': 0, 'Calcium': 30, 'Iron': 2, 'Potassium': 31}
https://www.allrecipes.com/recipe/240376/homemade-pepperoni-pizza/
{'search_term': 'pizza', 'name': 'Homemade Pepperoni Pizza', 'rating': '4.8', 'calories': '590', 'servings': '6', 'Total Fat': 29, 'Saturated Fat': 10, 'Cholesterol': 48, 'Sodium': 1408, 'Total Carbohydrate': 60, 'Dietary Fiber': 3, 'Total Sugars': 6, 'Protein': 20, 'Vitamin C': 0, 'Calcium': 168, 'Iron': 3, 'Potassium': 104}
https://www.allrecipes.com/recipe/273816/chicken-bacon-pesto-pizza/
{'search_term': 'pizza', 'name': 'Chicken Bacon Pesto Pizza', 'rating': None, 'calories': '567', 'servings': '4', 'Total Fat': 30, 'Saturated Fat':

https://www.allrecipes.com/recipe/80800/new-york-style-pizza/
{'search_term': 'pizza', 'name': 'New York Style Pizza', 'rating': '4.2', 'calories': '719', 'servings': '4', 'Total Fat': 37, 'Saturated Fat': 16, 'Cholesterol': 88, 'Sodium': 1832, 'Total Carbohydrate': 57, 'Dietary Fiber': 4, 'Total Sugars': 5, 'Protein': 40, 'Vitamin C': 7, 'Calcium': 1089, 'Iron': 5, 'Potassium': 469}
https://www.allrecipes.com/recipe/281536/chicken-fajita-pizza/
{'search_term': 'pizza', 'name': 'Chicken Fajita Pizza', 'rating': '4.0', 'calories': '326', 'servings': '8', 'Total Fat': 13, 'Saturated Fat': 6, 'Cholesterol': 45, 'Sodium': 790, 'Total Carbohydrate': 31, 'Dietary Fiber': 2, 'Total Sugars': 5, 'Protein': 20, 'Vitamin C': 26, 'Calcium': 226, 'Iron': 2, 'Potassium': 208}
https://www.allrecipes.com/recipe/256915/easy-pizza-in-a-pinch/
{'search_term': 'pizza', 'name': 'Easy Pizza in a Pinch', 'rating': '4.8', 'calories': '516', 'servings': '8', 'Total Fat': 32, 'Saturated Fat': 11, 'Cholesterol':

https://www.allrecipes.com/recipe/277118/pork-rind-pizza-crust/
{'search_term': 'pizza', 'name': 'Pork Rind Pizza Crust', 'rating': None, 'calories': '382', 'servings': '4', 'Total Fat': 30, 'Saturated Fat': 14, 'Cholesterol': 185, 'Sodium': 535, 'Total Carbohydrate': 3, 'Dietary Fiber': 0, 'Total Sugars': 1, 'Protein': 29, 'Vitamin C': 0, 'Calcium': 288, 'Iron': 1, 'Potassium': 105}
https://www.allrecipes.com/recipe/223481/the-real-chicago-deep-dish-pizza-dough/
{'search_term': 'pizza', 'name': 'The Real Chicago Deep Dish Pizza Dough', 'rating': '4.5', 'calories': '299', 'servings': '8', 'Total Fat': 14, 'Saturated Fat': 2, 'Sodium': 362, 'Total Carbohydrate': 37, 'Dietary Fiber': 2, 'Total Sugars': 1, 'Protein': 5, 'Calcium': 8, 'Iron': 2, 'Potassium': 73}
https://www.allrecipes.com/recipe/245593/pierogi-pizza/
{'search_term': 'pizza', 'name': 'Pierogi Pizza', 'rating': '4.7', 'calories': '462', 'servings': '8', 'Total Fat': 26, 'Saturated Fat': 12, 'Cholesterol': 42, 'Sodium': 592, 

...

Message: disconnected: unable to connect to renderer
  (failed to check if window was closed: disconnected: unable to connect to renderer)
  (Session info: chrome=106.0.5249.119)
Stacktrace:
Backtrace:
	Ordinal0 [0x00BA1ED3+2236115]
	Ordinal0 [0x00B392F1+1807089]
	Ordinal0 [0x00A466FD+812797]
	Ordinal0 [0x00A3798A+752010]
	Ordinal0 [0x00A377D8+751576]
	Ordinal0 [0x00A47B63+818019]
	Ordinal0 [0x00AA518A+1200522]
	Ordinal0 [0x00A918A6+1120422]
	Ordinal0 [0x00A6A73D+960317]
	Ordinal0 [0x00A6B71F+964383]
	GetHandleVerifier [0x00E4E7E2+2743074]
	GetHandleVerifier [0x00E408D4+2685972]
	GetHandleVerifier [0x00C32BAA+532202]
	GetHandleVerifier [0x00C31990+527568]
	Ordinal0 [0x00B4080C+1837068]
	Ordinal0 [0x00B44CD8+1854680]
	Ordinal0 [0x00B44DC5+1854917]
	Ordinal0 [0x00B4ED64+1895780]
	BaseThreadInitThunk [0x770DFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77957B5E+286]
	RtlGetAppContainerNamedObjectPath [0x77957B2E+238]

Ran into an error. Proceeding...
Message: disconnected: unable to conn

Message: disconnected: unable to connect to renderer
  (failed to check if window was closed: disconnected: unable to connect to renderer)
  (Session info: chrome=106.0.5249.119)
Stacktrace:
Backtrace:
	Ordinal0 [0x00BA1ED3+2236115]
	Ordinal0 [0x00B392F1+1807089]
	Ordinal0 [0x00A466FD+812797]
	Ordinal0 [0x00A3798A+752010]
	Ordinal0 [0x00A377D8+751576]
	Ordinal0 [0x00A47B63+818019]
	Ordinal0 [0x00AA518A+1200522]
	Ordinal0 [0x00A918A6+1120422]
	Ordinal0 [0x00A6A73D+960317]
	Ordinal0 [0x00A6B71F+964383]
	GetHandleVerifier [0x00E4E7E2+2743074]
	GetHandleVerifier [0x00E408D4+2685972]
	GetHandleVerifier [0x00C32BAA+532202]
	GetHandleVerifier [0x00C31990+527568]
	Ordinal0 [0x00B4080C+1837068]
	Ordinal0 [0x00B44CD8+1854680]
	Ordinal0 [0x00B44DC5+1854917]
	Ordinal0 [0x00B4ED64+1895780]
	BaseThreadInitThunk [0x770DFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77957B5E+286]
	RtlGetAppContainerNamedObjectPath [0x77957B2E+238]

Ran into an error. Proceeding...
Message: disconnected: unable to conn

Message: disconnected: unable to connect to renderer
  (failed to check if window was closed: disconnected: unable to connect to renderer)
  (Session info: chrome=106.0.5249.119)
Stacktrace:
Backtrace:
	Ordinal0 [0x00BA1ED3+2236115]
	Ordinal0 [0x00B392F1+1807089]
	Ordinal0 [0x00A466FD+812797]
	Ordinal0 [0x00A3798A+752010]
	Ordinal0 [0x00A377D8+751576]
	Ordinal0 [0x00A47B63+818019]
	Ordinal0 [0x00AA518A+1200522]
	Ordinal0 [0x00A918A6+1120422]
	Ordinal0 [0x00A6A73D+960317]
	Ordinal0 [0x00A6B71F+964383]
	GetHandleVerifier [0x00E4E7E2+2743074]
	GetHandleVerifier [0x00E408D4+2685972]
	GetHandleVerifier [0x00C32BAA+532202]
	GetHandleVerifier [0x00C31990+527568]
	Ordinal0 [0x00B4080C+1837068]
	Ordinal0 [0x00B44CD8+1854680]
	Ordinal0 [0x00B44DC5+1854917]
	Ordinal0 [0x00B4ED64+1895780]
	BaseThreadInitThunk [0x770DFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77957B5E+286]
	RtlGetAppContainerNamedObjectPath [0x77957B2E+238]

Ran into an error. Proceeding...
Message: disconnected: unable to conn

Message: disconnected: unable to connect to renderer
  (failed to check if window was closed: disconnected: unable to connect to renderer)
  (Session info: chrome=106.0.5249.119)
Stacktrace:
Backtrace:
	Ordinal0 [0x00BA1ED3+2236115]
	Ordinal0 [0x00B392F1+1807089]
	Ordinal0 [0x00A466FD+812797]
	Ordinal0 [0x00A3798A+752010]
	Ordinal0 [0x00A377D8+751576]
	Ordinal0 [0x00A47B63+818019]
	Ordinal0 [0x00AA518A+1200522]
	Ordinal0 [0x00A918A6+1120422]
	Ordinal0 [0x00A6A73D+960317]
	Ordinal0 [0x00A6B71F+964383]
	GetHandleVerifier [0x00E4E7E2+2743074]
	GetHandleVerifier [0x00E408D4+2685972]
	GetHandleVerifier [0x00C32BAA+532202]
	GetHandleVerifier [0x00C31990+527568]
	Ordinal0 [0x00B4080C+1837068]
	Ordinal0 [0x00B44CD8+1854680]
	Ordinal0 [0x00B44DC5+1854917]
	Ordinal0 [0x00B4ED64+1895780]
	BaseThreadInitThunk [0x770DFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77957B5E+286]
	RtlGetAppContainerNamedObjectPath [0x77957B2E+238]

Ran into an error. Proceeding...
Message: disconnected: unable to conn

Message: disconnected: unable to connect to renderer
  (failed to check if window was closed: disconnected: unable to connect to renderer)
  (Session info: chrome=106.0.5249.119)
Stacktrace:
Backtrace:
	Ordinal0 [0x00BA1ED3+2236115]
	Ordinal0 [0x00B392F1+1807089]
	Ordinal0 [0x00A466FD+812797]
	Ordinal0 [0x00A3798A+752010]
	Ordinal0 [0x00A377D8+751576]
	Ordinal0 [0x00A47B63+818019]
	Ordinal0 [0x00AA518A+1200522]
	Ordinal0 [0x00A918A6+1120422]
	Ordinal0 [0x00A6A73D+960317]
	Ordinal0 [0x00A6B71F+964383]
	GetHandleVerifier [0x00E4E7E2+2743074]
	GetHandleVerifier [0x00E408D4+2685972]
	GetHandleVerifier [0x00C32BAA+532202]
	GetHandleVerifier [0x00C31990+527568]
	Ordinal0 [0x00B4080C+1837068]
	Ordinal0 [0x00B44CD8+1854680]
	Ordinal0 [0x00B44DC5+1854917]
	Ordinal0 [0x00B4ED64+1895780]
	BaseThreadInitThunk [0x770DFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77957B5E+286]
	RtlGetAppContainerNamedObjectPath [0x77957B2E+238]

Ran into an error. Proceeding...


In [8]:
len(data)

2740

In [9]:
with open('burger_cookie_eggs_bread_sandwhich_taco_meatloaf', 'w') as f:
    json.dump(data, f, indent=4)